In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from fastDamerauLevenshtein import damerauLevenshtein
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tensorflow.keras import layers

In [35]:
df = pd.read_csv('data/single_step_df_ints_2022-04-14_encoded.csv', header=0)

In [36]:
df

,input,target,start_coords.x,start_coords.y,start_coords.z,h.already_seen,y.already_seen,e.already_seen,v.already_seen,o.already_seen,...,v.food_k,v.strong_k,v.mid_k,coordinates_y.x,coordinates_y.y,coordinates_y.z,y.containment,y.food_k,y.strong_k,y.mid_k
0,<start>,p,-0.451354,-0.413918,0.156247,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p,o,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,o,c,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c,g,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,g,k,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,<start>,d,1.000000,0.000000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2097,d,i,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2098,i,k,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2099,k,x,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
float_cols = df.select_dtypes(include=['float64']).columns
df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)

In [38]:
df['target'].unique()
df['target'].value_counts()

k    279
p    249
f    172
g    172
d    169
c    152
s    146
b    114
i    107
w     74
a     74
o     73
r     60
u     56
q     43
h     36
v     30
e     22
t     22
z     21
m     13
n      9
x      7
y      1
Name: target, dtype: int64

In [39]:
index_to_drop = df['target'].loc[df['target'] == 'y'].index[0]
index_to_drop

1972

In [40]:
nr_of_sequences = len(df[df['input'] == '<start>'])

In [41]:
# remove row with target that only occurs once

df_new = df.drop(index=index_to_drop, axis=0)
df_new.reset_index(inplace=True, drop=True)

In [42]:
# generate stratified split

split = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

for train_index, test_index in split.split(df_new[:1462], df_new[:1462]['target']):
    strat_train = df_new.loc[train_index]
    strat_test_val = df_new.loc[test_index]

In [45]:
# split test_val into test and val

split_test_val = len(strat_test_val)//2

strat_test = strat_test_val[:split_test_val]
strat_val = strat_test_val[split_test_val:]

In [46]:
# generate list of labels to pass to MultiLabelBinarizer so there's the same number of
# classes for all datasets

labels = df_new['target'].unique()

In [47]:
len(labels)

23

In [48]:
def create_dataset(dataframe, labels, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels_ds = df.pop('target')
    encoder = MultiLabelBinarizer(classes=labels)
    encoded_labels = encoder.fit_transform(labels_ds)
    
    df = {key: value[:, tf.newaxis] for key, value in df.items()}
    dataset = tf.data.Dataset.from_tensor_slices((dict(df), encoded_labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    
    return dataset

In [49]:
batch_size = 128

In [50]:
train_ds = create_dataset(strat_train, labels, batch_size=batch_size)
val_ds = create_dataset(strat_val, labels, shuffle=False, batch_size=batch_size)
test_ds = create_dataset(strat_test, labels, shuffle=False, batch_size=batch_size)

/tmp/ipykernel_21277/1510453973.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in df.items()}


In [15]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [16]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
        
    # prepare tf.data.Dataset that only yields the feature    
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # learn set of possible values and assign fixed int index
    index.adapt(feature_ds)
    
    # encode int indices
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # apply multi-hot encoding to indices
    # lambda function captures the layer to include them in Keras functional models later
    return lambda feature: encoder(index(feature))

In [17]:
def create_input_data(dataframe):
    all_inputs = []
    encoded_features = []
    
    for header in dataframe.columns:
        # numerical features
        if 'coord' in header or 'already' in header:
            numeric_col = tf.keras.Input(shape=(1,), name=header)
            normalization_layer = get_normalization_layer(header, train_ds)
            encoded_numeric_col = normalization_layer(numeric_col)
            all_inputs.append(numeric_col)
            encoded_features.append(encoded_numeric_col)
        
        # categorical features
        elif 'containment' in header or 'food' in header or 'mid' in header or \
        'strong' in header:
            categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
            encoding_layer = get_category_encoding_layer(name=header,
                                                        dataset=train_ds,
                                                        dtype='int64')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
        elif header == 'input':
            categorical_col = tf.keras.Input(shape=(1,), name='input', dtype='string')
            encoding_layer = get_category_encoding_layer(name='input',
                                                        dataset=train_ds,
                                                        dtype='string')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
    return all_inputs, encoded_features

In [51]:
all_inputs, encoded_features = create_input_data(df)

In [52]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [53]:
# use model from tutorial with dense layers

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(512, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(len(labels))(x)

model = tf.keras.Model(all_inputs, output)

In [54]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [81]:
#tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

In [55]:
model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=[callback])

Epoch 1/100
8/8 [==============================] - 7s 403ms/step - loss: 2.9932 - accuracy: 0.0968 - val_loss: 2.3648 - val_accuracy: 0.1591
Epoch 2/100
8/8 [==============================] - 0s 53ms/step - loss: 2.4354 - accuracy: 0.1887 - val_loss: 2.1287 - val_accuracy: 0.2773
Epoch 3/100
8/8 [==============================] - 0s 51ms/step - loss: 2.2303 - accuracy: 0.2219 - val_loss: 2.0207 - val_accuracy: 0.3045
Epoch 4/100
8/8 [==============================] - 0s 53ms/step - loss: 2.1484 - accuracy: 0.2522 - val_loss: 1.9593 - val_accuracy: 0.3364
Epoch 5/100
8/8 [==============================] - 0s 53ms/step - loss: 2.0193 - accuracy: 0.2913 - val_loss: 1.8758 - val_accuracy: 0.3864
Epoch 6/100
8/8 [==============================] - 0s 51ms/step - loss: 1.9105 - accuracy: 0.3069 - val_loss: 1.8195 - val_accuracy: 0.4045
Epoch 7/100
8/8 [==============================] - 0s 52ms/step - loss: 1.8419 - accuracy: 0.3490 - val_loss: 1.7832 - val_accuracy: 0.3909
Epoch 8/100
8/8 [==

8/8 [==============================] - 0s 52ms/step - loss: 0.9414 - accuracy: 0.6549 - val_loss: 1.3365 - val_accuracy: 0.5364
Epoch 60/100
8/8 [==============================] - 0s 52ms/step - loss: 0.9680 - accuracy: 0.6530 - val_loss: 1.3383 - val_accuracy: 0.5455
Epoch 61/100
8/8 [==============================] - 0s 52ms/step - loss: 0.9414 - accuracy: 0.6510 - val_loss: 1.3464 - val_accuracy: 0.5091
Epoch 62/100
8/8 [==============================] - 0s 51ms/step - loss: 0.9404 - accuracy: 0.6608 - val_loss: 1.3612 - val_accuracy: 0.5182
Epoch 63/100
8/8 [==============================] - 0s 52ms/step - loss: 0.9243 - accuracy: 0.6637 - val_loss: 1.3586 - val_accuracy: 0.5136
Epoch 64/100
8/8 [==============================] - 0s 53ms/step - loss: 0.9370 - accuracy: 0.6725 - val_loss: 1.3620 - val_accuracy: 0.5364
Epoch 65/100
8/8 [==============================] - 0s 52ms/step - loss: 0.9164 - accuracy: 0.6706 - val_loss: 1.3639 - val_accuracy: 0.5273
Epoch 66/100
8/8 [========

In [56]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

2/2 [==============================] - 0s 36ms/step - loss: 4057.2468 - accuracy: 0.5251
Accuracy 0.5251141786575317


In [57]:
model.save('models/next_obj_classifier_tablesetting_2022-05-10')

INFO:tensorflow:Assets written to: models/next_obj_classifier_tablesetting_2022-05-10/assets


INFO:tensorflow:Assets written to: models/next_obj_classifier_tablesetting_2022-05-10/assets


In [58]:
reloaded_model = tf.keras.models.load_model('models/next_obj_classifier_tablesetting_2022-05-10/')

In [ ]:
# test prediction for one sample

In [27]:
sample = df.loc[0].drop('target').to_dict()

In [28]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [29]:
prediction = reloaded_model.predict(input_dict)
prediction = tf.nn.sigmoid(prediction[0])

In [30]:
# get label for prediction

pred_label = labels[np.argmax(prediction)]
pred_label

'p'

In [72]:
def get_prequential_error(dataframe, model, labels, nr_of_sequences):
    errors = [[] for seq in range(0, nr_of_sequences + 1)]
    start_token_count = 0
    sequence_nr = 0
    
    for row in range(dataframe.index[0], dataframe.index[-1]): 
        observed_target = dataframe.loc[row, 'target']
        sample = dataframe.loc[row].drop('target').to_dict()
        input_dict = {name: tf.convert_to_tensor([value]) for name, value in 
                          sample.items()}
        predicted_target = model.predict(input_dict)
        predicted_target = tf.nn.sigmoid(predicted_target[0])
            
        pred_label = labels[np.argmax(predicted_target)]
        error = 1 - damerauLevenshtein(pred_label, observed_target)
        errors[sequence_nr].append(error)
        
        if row != 0 and dataframe.loc[row, 'input'] == '<start>':
            start_token_count += 1
        
        if start_token_count > 0:
            sequence_nr += 1
            start_token_count = 0
            
    return errors

In [71]:
df[1461:].index[-1]

2100

In [73]:
errors = get_prequential_error(df.iloc[1461:], reloaded_model, labels, nr_of_sequences)

In [74]:
summed_error = [sum(error) for error in errors[:-1]]

In [75]:
np.median(summed_error)

0.0

In [78]:
#with open('results/nn_spatialinfo_prequential_summed_fitted_on_ts_results_cooking_2022-05-10.txt', 'w') as file:
#    file.write(str(summed_error))

In [77]:
np.mean(summed_error)

1.6560509554140128